Notebook to parse the qsbio and pdbseqres data for ProtTrans embeddings 



In [ ]:
# Function definitions 
def code_mod(code_string):

    pdb_code = str(code_string.split("_")[0])
    modified_code = pdb_code
    return modified_code

In [ ]:
# install Biopython in case it doesn't exist already
!pip install Bio

# imports 
import os 
import os.path
import pandas as pd 
import Bio
from Bio import SeqIO
import pickle 
from google.colab import drive
drive.mount("/content/drive")

# relevant paths for this project
Dir = "drive/MyDrive/OrlyPred/"
Data = "drive/MyDrive/OrlyPred/Data"
qsbio_file = "drive/MyDrive/OrlyPred/Data/QSbio_PiQSi_annotations_V6_2020.csv"

     |████████████████████████████████| 271 kB 24.3 MB/s 
     |████████████████████████████████| 2.3 MB 49.6 MB/s 
Mounted at /content/drive


In [ ]:
# read qsbio data into a df 
qsbio_df = pd.read_csv(qsbio_file, error_bad_lines=False, low_memory=False, skiprows=21)
qsbio_df = qsbio_df.drop(["Unnamed: 18", "Unnamed: 19", "Unnamed: 20", "Unnamed: 21"], axis=1)

# qsbio_df.head()

# keep only pdbID, subunits columns and homomer indication
qsbio_filt = qsbio_df[['code', 'nsub', 'corrected_nsub','QSBIO_err_prob', 'best_BU', 'homo']]

# remove rows with Nan values of nsub
# removes 12210 rows
qsbio_filt.dropna(subset=["nsub"], inplace=True)

# fill empty corrected_nsub with the val from nsub
qsbio_filt['corrected_nsub'].fillna(qsbio_filt['nsub'], inplace=True)

# create a new column with just pdb id
qsbio_filt['pdb_code'] = qsbio_filt['code'].apply(lambda x: code_mod(x))

# summarize how many instances are unique for each pdb code
# print(qsbio_filt.groupby('pdb_code').nunique())

# for each pdb code, retain only rows that have a unique nsub value
qsbio_uniq = qsbio_filt.drop_duplicates(subset=['pdb_code','nsub'],keep="last")

#store relevant cols as ints
qsbio_uniq["homo"] = pd.to_numeric(qsbio_uniq['homo'], errors='coerce', downcast="integer")
qsbio_uniq["corrected_nsub"] = pd.to_numeric(qsbio_uniq['corrected_nsub'], errors='coerce', downcast="integer")
qsbio_uniq["nsub"] = pd.to_numeric(qsbio_uniq['nsub'], errors='coerce', downcast="integer")
qsbio_uniq["QSBIO_err_prob"] = pd.to_numeric(qsbio_uniq['QSBIO_err_prob'], errors='coerce', downcast="integer")
qsbio_uniq["best_BU"] = pd.to_numeric(qsbio_uniq['best_BU'], errors='coerce', downcast="integer")


# retain only homomers, i.e. rows where homo is 1 (from the documentation: '1' indicates homomers, otherwise heteromers)
qsbio_homomer = qsbio_uniq[qsbio_uniq["homo"] == 1]

# the next rows are meant for data filtration 
# filter by QSbio error estimator as recommended by Emmanuel Levy
qsbio_df_conf = qsbio_homomer[qsbio_homomer["QSBIO_err_prob"] < 15]
# An additional confidence value - the best Biological Unit of the strcuture
qsbio_df_conf_plus = qsbio_df_conf[qsbio_df_conf["best_BU"] == 1]
# take only rows where the nsub and corrected_nsub are the same (removes less than 2000 structures)
qsbio_homomer_final = qsbio_df_conf_plus[qsbio_df_conf_plus['corrected_nsub'] == qsbio_df_conf_plus['nsub']]
qsbio_homomer_final.drop(["corrected_nsub"], axis=1, inplace=True)


print(qsbio_homomer_final.head(20))

In [ ]:
print(qsbio_homomer_final.groupby('pdb_code').size().count())

56798


In [ ]:
# qsbio_uniq[qsbio_uniq["pdb_code"] == "6hdt"]

In [ ]:
# qsbio_homomer_final[qsbio_homomer_final["pdb_code"] == "6hdv"]

In [ ]:
print(qsbio_df.shape)
print(qsbio_filt.shape)
print(qsbio_uniq.shape)
print(qsbio_homomer.shape)
print(qsbio_homomer_final.shape)

# (210680, 18)
# (198470, 7)
# (137690, 7)
# (119129, 7)
# (56798, 6)


In [ ]:
# save a list of unique pdb ids for fasta retrieval  
# pdb_ids = qsbio_filt.pdb_code.unique()
pdb_ids_homo = qsbio_homomer_final.pdb_code.unique()

# print(pdb_ids.shape)
print(pdb_ids_homo.shape)
print(pdb_ids_homo)

(56798,)
['101m' '102l' '102m' ... '9rub' '9xia' '9xim']


In [ ]:
#!head $Data/pdb_seqres.txt > $Data/test.fasta
#test_path = os.path.join(Data,"test.fasta")
test_path = os.path.join(Data,"pdb_seqres.txt")
#print(test_path)

#records = list(SeqIO.parse(test_path,"fasta"))
record_dict = SeqIO.to_dict(SeqIO.parse(test_path,"fasta"))
#print(record_dict)

pdb_dict = dict()
for k in record_dict.keys():
  pdb_dict.update({k[0:4]:pdb_dict.get(k[0:4],list())+[k]})
  #print(k)
  # pdb_dict[k[0:4]].update(pdb_dict.get(k[0:4],list()).append(k))

In [ ]:
# print the dict to see that it works 
count = 0
for k,v in pdb_dict.items():
  print(k,v)
  count +=1
  if count == 10:
    break

101m ['101m_A']
102l ['102l_A']
102m ['102m_A']
103l ['103l_A']
103m ['103m_A']
104l ['104l_A', '104l_B']
104m ['104m_A']
105m ['105m_A']
106m ['106m_A']
107l ['107l_A']


In [ ]:
# even though we are working only with homomers we need this selection
# since homomoric stuctures that contain peptides are still noted as homomoers 
# but practically have several different chains (protein and peptide(s))

def select_fasta(pdb_id):
  i_val = pdb_dict.get(pdb_id, None)
  if i_val is None:
    return None 
  if len(i_val)==1:
    return " ".join(str(record_dict[i_val[0]].seq))
  else: 
    longest_fasta = record_dict[i_val[0]].seq
    for i_fasta in i_val[1:]:
      if len(record_dict[i_fasta].seq) > len(longest_fasta):
        longest_fasta = record_dict[i_fasta].seq
  
    return " ".join(longest_fasta)

fasta_list = [select_fasta(x) for x in pdb_ids_homo]
qsbio_homomer_final['fasta'] = qsbio_homomer_final.pdb_code.apply(select_fasta)
# print(qsbio_homomer_final.head())
# print(fasta_list[:5])



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
qsbio_homomer_final.shape


(56798, 7)

In [ ]:
# qsbio_homomer_final[qsbio_homomer_final["pdb_code"] == "4z28"]
print(qsbio_homomer_final.groupby('fasta').size().count())
qsbio_homomer_final.groupby("fasta").size().sort_values(ascending=False)

31994


fasta
S T G S A T T T P I D S L D D A Y I T P V Q I G T P A Q T L N L D F D T G S S D L W V F S S E T T A S E V D G Q T I Y T P S K S T T A K L L S G A T W S I S Y G D G S S S S G D V Y T D T V S V G G L T V T G Q A V E S A K K V S S S F T E D S T I D G L L G L A F S T L N T V S P T Q Q K T F F D N A K A S L D S P V F T A D L G Y H A P G T Y N F G F I D T T A Y T G S I T Y T A V S T K Q G F W E W T S T G Y A V G S G T F K S T S I D G I A D T G T T L L Y L P A T V V S A Y W A Q V S G A K S S S S V G G Y V F P C S A T L P S F T F G V G S A R I V I P G D Y I D F G P I S T G S S S C F G G I Q S S A G I G I N I F G D V A L K A A F V V F N G A T T P T L G F A S K                                                                                                                                                            518
K V F G R C E L A A A M K R H G L D N Y R G Y S L G N W V C A A K F E S N F N T Q A T N R N T D G S T D Y G I L Q I N S R W W C N D G R T P G S R N L C N I P C S A L L S S D I

In [ ]:

# remove sequences where no fasta was found 
qsbio_no_none_fasta = qsbio_homomer_final[qsbio_homomer_final['fasta'].notnull()]

# remove redundant fasta entries (100% seq. id), keep the highest conf. entry
qsbio_no_none_fasta = qsbio_no_none_fasta.sort_values(["QSBIO_err_prob", "fasta"], ascending=True).drop_duplicates(subset=["fasta"])

with open("drive/MyDrive/OrlyPred/Homomer_embeds/results/embeds_Mar_22/parsed_tab_for_embed.pkl", 'wb') as f:
  pickle.dump(qsbio_no_none_fasta, f)

qsbio_no_none_fasta